# LLaMa-Alpaca-LoRA 파인튜닝

나만의 데이터로 ChatGPT 만들기

> 유튜브 [빵형의 개발도상국](https://www.youtube.com/@bbanghyong)

![](https://github.com/kairess/alpaca-lora/raw/main/result.jpg)

In [1]:
!nvidia-smi

Mon Apr 22 07:41:49 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla V100-SXM2-16GB           Off | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P0              27W / 300W |      0MiB / 16384MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

## Download source code and install dependencies

- https://github.com/tloen/alpaca-lora.git
- 주의!: bitsandbytes가 현재 Linux에서만 동작함
- 주의!!: peft 버그 https://github.com/tloen/alpaca-lora/issues/293

In [2]:
!git clone -q https://github.com/kairess/alpaca-lora.git
%cd alpaca-lora
!pip install -r requirements.txt -q
!pip uninstall peft -y -q     # 모델의 모든 파라미터를 미세 조정하지 않고도 사전 학습된 언어 모델을 효율적으로 적용가능
!pip install -q git+https://github.com/huggingface/peft.git@e536616888d51b453ed354a6f1e243fecb02ea08

/content/alpaca-lora
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 79.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 51.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.4/88.4 kB 12.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 82.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 25.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.9/388.9 kB 39.7

## Dataset

### - 49,620개의 Instruction, Input, Response set.

- https://github.com/Beomi/KoAlpaca/blob/main/ko_alpaca_data.json
- https://huggingface.co/datasets/Bingsu/ko_alpaca_data

### - shareGPT 한국어 번역 데이터셋

이 프로젝트는 shareGPT 데이터셋 60만 대화문을 DeepL 을 통해 한국어로 번역하고 있습니다.

https://huggingface.co/datasets/junelee/sharegpt_deepl_ko

```json
[
  {
    "instruction":"직원들의 급여를 기록하는 데이터베이스를 설계하십시오.",
    "input":"",
    "output":"직원들의 급여를 새로운 데이터베이스에서 추적하려면, 다음과 같은 열이 포함된 \"Employee\" 테이블을 만들어야 합니다: \"name\", \"salary\", \"start_date\", \"end_date\"."
  },
  {
    "instruction":"아래 문장의 각 단어에 품사를 지정하십시오.",
    "input":"새끼 고양이는 종종 신나게 뛰어다니기도 합니다.",
    "output":"새끼(Noun) 고양이(Noun) 는 은 종종 Adverb 신나게 Adverb 뛰어다니기도 합니다."
  },
  ...
]
```

In [3]:
!wget -q https://github.com/Beomi/KoAlpaca/raw/main/ko_alpaca_data.json

import json

with open('ko_alpaca_data.json', 'r') as f:
    dataset = json.load(f)

dataset[:5]

[{'instruction': '건강을 유지하기 위한 세 가지 팁을 알려주세요.',
  'input': '',
  'output': '세 가지 팁은 아침식사를 꼭 챙기며, 충분한 수면을 취하고, 적극적으로 운동을 하는 것입니다.'},
 {'instruction': '세 가지 기본 색은 무엇인가요?',
  'input': '',
  'output': '기본 색은 빨강, 파랑, 노랑입니다.'},
 {'instruction': '원자의 구조를 설명하세요.',
  'input': '',
  'output': '원자는 양성자, 중성자, 전자로 구성되어 있으며, 양성자와 중성자는 원자핵 안에 있고 전자는 주변에 있습니다.'},
 {'instruction': '대기 오염을 어떻게 줄일 수 있나요?',
  'input': '',
  'output': '대기 오염을 줄이기 위해 자동차 대신 대중교통을 이용하고, 미세먼지 배출량을 줄이기 위해 차량 정기 검사를 받아야 합니다. 또한 에너지 절약을 위해 전기를 아끼고, 나무를 심어 대기 오염을 줄일 수 있습니다.'},
 {'instruction': '어려운 결정을 내려야 했던 때를 설명하세요.',
  'input': '',
  'output': '제가 어려운 결정을 내려야 했던 때는 대학원 졸업 후 직장을 찾아야 했던 때입니다. 다양한 선택지가 있었지만 나의 관심사와 비전에 가장 부합하는 경력을 뽑는데 어려움이 있었습니다. 이 때 여러 사람들의 조언과 현재 상황에 대한 깊은 고민 끝에 적절한 선택을 내리게 되었습니다.'}]

## Finetune

기본 설정으로 했을 때:
- 3h/1epoch in NVIDIA A100
- 12h/1epoch in NVIDIA T4

### Create a custom template

Default Alpaca

```json
{
    "description": "Template used by Alpaca-LoRA.",
    "prompt_input": "Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\n{instruction}\n\n### Input:\n{input}\n\n### Response:\n",
    "prompt_no_input": "Below is an instruction that describes a task. Write a response that appropriately completes the request.\n\n### Instruction:\n{instruction}\n\n### Response:\n",
    "response_split": "### Response:"    
}
```

In [4]:
import json

prompt_template = {
    "description": "Alpaca-LoRA Custom 템플릿",
    "prompt_input": (
        "Below is an instruction that describes a task, paired with an input that provides further context.\n"
        "아래는 작업을 설명하는 명령어와 추가적 맥락을 제공하는 입력이 짝을 이루는 예제입니다.\n\n"
        "Write a response that appropriately completes the request.\n요청을 적절히 완료하는 응답을 작성하세요.\n\n"
        "### Instruction(명령어):\n{instruction}\n\n### Input(입력):\n{input}\n\n### Response:\n"
    ),
    "prompt_no_input": (
        "Below is an instruction that describes a task.\n"
        "아래는 작업을 설명하는 명령어입니다.\n\n"
        "Write a response that appropriately completes the request.\n명령어에 따른 요청을 적절히 완료하는 응답을 작성하세요.\n\n"
        "### Instruction(명령어):\n{instruction}\n\n### Response:\n"
    ),
    "response_split": "### Response:",
}

with open('templates/custom.json', 'w', encoding='utf-8') as f:
    json.dump(prompt_template, f, ensure_ascii=False)

### Hyperparameters

https://github.com/tloen/alpaca-lora/blob/0e1a5d52a460d14aea2325e43c302972badb9cdd/finetune.py#L28

In [ ]:
!python finetune.py \
    --base_model 'decapoda-research/llama-7b-hf' \
    --data_path 'ko_alpaca_data.json' \
    --output_dir './output' \
    --num_epochs 5 \
    --learning_rate 5e-4 \
    --val_set_size 2000 \
    --batch_size 512 \
    --micro_batch_size 16 \
    --prompt_template_name 'custom'

2023-04-17 13:17:53.394813: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT

===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /usr/local/lib/python3.9/dist-packages/bitsandbytes/libbitsandbytes_cuda118_nocublaslt.so
/usr/local/lib/python3.9/dist-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: /usr/lib64-nvidia did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)
/usr/local/lib/python3.9/dist-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/sys/fs/cgroup/memory.events /var/colab/cgroup/jupyter-children/memory.events')}
  war

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
!mkdir /content/drive/MyDrive/LLaMa-Alpaca-LoRA
!cp -a output /content/drive/MyDrive/LLaMa-Alpaca-LoRA

Mounted at /content/drive


## Test on Gradio

--load_8bit

In [ ]:
!python generate.py \
    --base_model 'decapoda-research/llama-7b-hf' \
    --lora_weights '/content/drive/MyDrive/LLaMa-Alpaca-LoRA/output' \
    --prompt_template 'custom' \
    --share_gradio

2023-04-17 23:28:10.441532: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT

===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /usr/local/lib/python3.9/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so
/usr/local/lib/python3.9/dist-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: /usr/lib64-nvidia did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)
/usr/local/lib/python3.9/dist-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/sys/fs/cgroup/memory.events /var/colab/cgroup/jupyter-children/memory.events')}
  warn(msg)
/usr